<a href="https://colab.research.google.com/github/woonghee97/deep_learning/blob/DeepLearning/HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HW#6 Regularization

안녕하세요, 광운대학교 로봇학부의 오정현 교수입니다. 본 자료는 딥러닝 실습 수업을 위해 제작된 것입니다.

파이썬 문법
- 점프투파이썬(https://wikidocs.net/book/1) 참고

이번 과제는 딥러닝의 일반화 성능을 높이기 위한 Regularization을 해보는 것입니다.이미지 분류에 여러 가지 Regularization 기법을 적용해 보도록 하겠습니다. 대표적인 Regularization 기법으로 Dropout, Data augmentation, Batch Normalization 등이 있습니다.

이번 과제는 (https://www.tensorflow.org/tutorials/keras/classification?hl=ko)를 참고하면 좋습니다.

#1. Data Generation
Data는 mnist dataset을 이용하도록 하겠습니다. mnist dataset은 원래 60000개의 training set이 주어져 있지만 overfitting을 유도하기 위하여 1000개의 data만 이용하려고 합니다. 1000개의 data로 이루어진 x_train과 y_train을 만들어 보세요. 그리고 2000개로 이루어진 large_x_train, large_y_train을 만들어보세요. 그리고 training data의 다른 범위에서 200개로 이루어진 x_validation과 y_validation도 만들어 보세요.

In [7]:
from __future__ import print_function
import tensorflow as tf
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Activation, Flatten
from keras import backend as K
from matplotlib import pyplot
from keras.utils import np_utils

batch_size = 28
num_classes = 10
epochs = 100

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(X_train, Y_train), (x_test,y_test) = mnist.load_data()

### START CODE HERE ###
x_validation = X_train[3000:3200]
y_validation = Y_train[3000:3200]
large_x_train = X_train[0:2000]
large_y_train = Y_train[0:2000]
x_train = X_train[2000:3000]
y_train = Y_train[2000:3000]
### END CODE HERE ###
print("x_validation shape:", x_validation.shape)
print("y_validation shape:", y_validation.shape)
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    large_x_train = large_x_train.reshape(large_x_train.shape[0], 1, img_rows, img_cols)
    x_validation = x_validation.reshape(x_validation.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    large_x_train = large_x_train.reshape(large_x_train.shape[0], img_rows, img_cols, 1)
    x_validation = x_validation.reshape(x_validation.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
large_x_train = large_x_train.astype('float32')
x_validation = x_validation.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
large_x_train /= 255
x_validation /= 255
x_test /= 255

# convert class vectors to binary class matrices
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
large_y_train = keras.utils.np_utils.to_categorical(large_y_train, num_classes)
y_validation = keras.utils.np_utils.to_categorical(y_validation, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

assert large_x_train.shape[0]==2000
assert large_y_train.shape[0]==2000
assert x_train.shape[0]==1000
assert y_train.shape[0]==1000
assert x_validation.shape[0]==200
assert y_validation.shape[0]==200

print("x_train shape:", x_train.shape)
print("y_train shape:", y_train.shape)
print("large_x_train shape:", large_x_train.shape)
print("large_y_train shape:", large_y_train.shape)
print("x_validation shape:", x_validation.shape)
print("y_validation shape:", y_validation.shape)

x_validation shape: (200, 28, 28)
y_validation shape: (200,)
x_train shape: (1000, 28, 28, 1)
y_train shape: (1000, 10)
large_x_train shape: (2000, 28, 28, 1)
large_y_train shape: (2000, 10)
x_validation shape: (200, 28, 28, 1)
y_validation shape: (200, 10)


#2. 모델 생성
복습 차원에서 MLP 분류모델을 만들어 보도록 하겠습니다. 모델의 마지막 레이어에는 활성화 함수로 10개의 출력을 가지는 softmax를 달겠습니다. 이를 통해서 모델은 이미지안의 숫자가 0부터 9까지의 숫자중에 어디에 가까운지를 확률적으로 나타냅니다. 



다음과 같은 MLP 모델을 만들어 보세요.

| Layer (type) | Output Shape | Param # |
|------|------|------|
| Flatten | (None, 784) | 0 |
| Dense | (None, 1024) | 803840 |
| Dense | (None, 1024) | 1049600 |
| Dense | (None, 1024) | 1049600 |
| Flatten | (None, 1024) | 0 |
| Dense | (None, 10) | 10250 |

In [2]:
model = Sequential()

### START CODE HERE ###
model.add(Flatten())
model.add(Dense(1024))
model.add(Dense(1024))
model.add(Dense(1024))
model.add(Flatten())
model.add(Dense(10, activation='softmax'))
model.build(input_shape=(None, 28, 28, 1))
### END CODE HERE ###

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 1024)              803840    
                                                                 
 dense_1 (Dense)             (None, 1024)              1049600   
                                                                 
 dense_2 (Dense)             (None, 1024)              1049600   
                                                                 
 flatten_1 (Flatten)         (None, 1024)              0         
                                                                 
 dense_3 (Dense)             (None, 10)                10250     
                                                                 
Total params: 2,913,290
Trainable params: 2,913,290
Non-

#3. Learning MLP
기본 MLP 분류모델을 학습해 보겠습니다. Overfitting은 Training data에 맞추어 과도하게 학습이 이루어져 Test data에서 높은 성능이 나지 않는 현상, 즉 Generalization 성능이 높지 않게 나타나는 현상을 의미합니다. 따라서 Overfitting이 발생하면 Training accuracy는 높지만 Test accuracy는 높지 않게 나타납니다. 

현재 모델이 overfitting이 발생하는지 체크해 보세요. 일부러 overfitting이 발생하도록 유도하였기 때문에 overfitting 현상이 나타나야 합니다.

In [8]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])
weights = model.get_weights()

history=model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_validation, y_validation))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Epoch 1/100
36/36 [==============================] - 2s 40ms/step - loss: 2.3950 - accuracy: 0.0600 - val_loss: 2.3678 - val_accuracy: 0.0650
Epoch 2/100
36/36 [==============================] - 1s 34ms/step - loss: 2.2975 - accuracy: 0.0990 - val_loss: 2.2908 - val_accuracy: 0.1200
Epoch 3/100
36/36 [==============================] - 1s 34ms/step - loss: 2.2083 - accuracy: 0.1520 - val_loss: 2.2191 - val_accuracy: 0.1650
Epoch 4/100
36/36 [==============================] - 1s 35ms/step - loss: 2.1254 - accuracy: 0.2500 - val_loss: 2.1518 - val_accuracy: 0.2400
Epoch 5/100
36/36 [==============================] - 1s 34ms/step - loss: 2.0473 - accuracy: 0.3610 - val_loss: 2.0883 - val_accuracy: 0.3050
Epoch 6/100
36/36 [==============================] - 1s 34ms/step - loss: 1.9735 - accuracy: 0.4420 - val_loss: 2.0276 - val_accuracy: 0.3700
Epoch 7/100
36/36 [==============================] - 1s 34ms/step - loss: 1.9042 - accuracy: 0.5090 - val_loss: 1.9703 - val_accuracy: 0.4450
Epoch 

#4. Regularization
Overfitting이 발생한 모델에 다양한 Regularization 기법을 이용해 보도록 합시다.

4.1 Large Dataset

Training data가 충분하다면 overfitting 현상이 발생할 가능성이 줄어듭니다. 기본 MLP 분류 모델에서 large_x_train과 large_y_train을 이용하면 성능이 올라갈 것입니다. Generalization 성능이 올라갔나요?

In [9]:
#initialize weights
model.set_weights(weights)

large_model_history=model.fit(large_x_train, large_y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_validation, y_validation))

score = model.evaluate(x_test, y_test, verbose=0)
print('Largemodel Test loss:', score[0])
print('Largemodel Test accuracy:', score[1])

Epoch 1/100
72/72 [==============================] - 2s 34ms/step - loss: 2.3104 - accuracy: 0.1080 - val_loss: 2.2287 - val_accuracy: 0.1700
Epoch 2/100
72/72 [==============================] - 2s 34ms/step - loss: 2.0933 - accuracy: 0.3130 - val_loss: 2.0610 - val_accuracy: 0.3550
Epoch 3/100
72/72 [==============================] - 2s 34ms/step - loss: 1.9249 - accuracy: 0.4865 - val_loss: 1.9243 - val_accuracy: 0.4950
Epoch 4/100
72/72 [==============================] - 2s 34ms/step - loss: 1.7858 - accuracy: 0.5935 - val_loss: 1.8085 - val_accuracy: 0.5700
Epoch 5/100
72/72 [==============================] - 2s 34ms/step - loss: 1.6675 - accuracy: 0.6465 - val_loss: 1.7081 - val_accuracy: 0.6350
Epoch 6/100
72/72 [==============================] - 2s 34ms/step - loss: 1.5657 - accuracy: 0.6820 - val_loss: 1.6196 - val_accuracy: 0.6450
Epoch 7/100
72/72 [==============================] - 2s 33ms/step - loss: 1.4759 - accuracy: 0.7050 - val_loss: 1.5408 - val_accuracy: 0.6550
Epoch 

##4.2 Dropout
Dropout은 쉽게 쓸 수 있는 Regularization 기법입니다. Layer 사이에 Dropout layer만 추가하면 되기 때문에 간편합니다. 다음과 같은 Dropout model을 만들어 보세요.

| Layer (type) | Output Shape | Param # |
|------|------|------|
| Flatten | (None, 784) | 0 |
| Dense | (None, 1024) | 803840 |
| Dense | (None, 1024) | 1049600 |
| Dense | (None, 1024) | 1049600 |
| Dropout | (None, 1024) | 0 |
| Flatten | (None, 1024) | 0 |
| Dense | (None, 10) | 10250 |

In [15]:
dropout_model = Sequential()
### START CODE HERE ###
dropout_model.add(Flatten())
dropout_model.add(Dense(1024))
dropout_model.add(Dense(1024))
dropout_model.add(Dense(1024))
dropout_model.add(Dropout(0.2))
dropout_model.add(Flatten())
dropout_model.add(Dense(10, activation='softmax'))

dropout_model.build(input_shape=(None, 28, 28, 1))
### END CODE HERE ###

# ### START CODE HERE ###
# model.add(Flatten())
# model.add(Dense(1024))
# model.add(Dense(1024))
# model.add(Dense(1024))
# model.add(Flatten())
# model.add(Dense(10, activation='softmax'))
# model.build(input_shape=(None,784))
# ### END CODE HERE ###

dropout_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_8 (Flatten)         (None, 784)               0         
                                                                 
 dense_16 (Dense)            (None, 1024)              803840    
                                                                 
 dense_17 (Dense)            (None, 1024)              1049600   
                                                                 
 dense_18 (Dense)            (None, 1024)              1049600   
                                                                 
 dropout_3 (Dropout)         (None, 1024)              0         
                                                                 
 flatten_9 (Flatten)         (None, 1024)              0         
                                                                 
 dense_19 (Dense)            (None, 10)               

Dropout Model을 학습해 보겠습니다. Generalization 성능이 올라갔나요?

※Dropout을 적용한 Model은 일정확률로 신경망의 뉴런을 비활성화시키기 때문에, 오버피팅을 방지하는 효과가 있습니다. 하지만 비활성화로 인해서  학습속도가 떨어진다는 단점이 존재합니다. 그 때문에 test accuracy는 같은 학습파라미터 조건에서 기존모델보다 더 낮을 수 있습니다. 

기존모델과 dropout을 적용한 모델에 대해서, train 데이터와 test데이터에 대한 accuracy차이를 주목해보시면 좋을 것같습니다!

In [16]:
dropout_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

dropout_model_history=dropout_model.fit(large_x_train, large_y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_validation, y_validation))

score = dropout_model.evaluate(x_test, y_test, verbose=0)
print('Dropout model Test loss:', score[0])
print('Dropout model Test accuracy:', score[1])

Epoch 1/100
72/72 [==============================] - 3s 36ms/step - loss: 2.2521 - accuracy: 0.1930 - val_loss: 2.1332 - val_accuracy: 0.2650
Epoch 2/100
72/72 [==============================] - 2s 34ms/step - loss: 2.1105 - accuracy: 0.2630 - val_loss: 2.0125 - val_accuracy: 0.3400
Epoch 3/100
72/72 [==============================] - 2s 34ms/step - loss: 1.9908 - accuracy: 0.3560 - val_loss: 1.9073 - val_accuracy: 0.4000
Epoch 4/100
72/72 [==============================] - 2s 34ms/step - loss: 1.8789 - accuracy: 0.4305 - val_loss: 1.8135 - val_accuracy: 0.4800
Epoch 5/100
72/72 [==============================] - 2s 34ms/step - loss: 1.7828 - accuracy: 0.5025 - val_loss: 1.7286 - val_accuracy: 0.5450
Epoch 6/100
72/72 [==============================] - 2s 34ms/step - loss: 1.6902 - accuracy: 0.5465 - val_loss: 1.6500 - val_accuracy: 0.6000
Epoch 7/100
72/72 [==============================] - 2s 33ms/step - loss: 1.6060 - accuracy: 0.5945 - val_loss: 1.5790 - val_accuracy: 0.6450
Epoch 

##4.3 BatchNormalization
BatchNormalization(BN)은 쉽게 쓸 수 있는 Regularization 기법입니다. Layer 사이에 BN layer만 추가하면 되기 때문에 간편합니다. 다음과 같은 BN model을 만들어 보세요.

| Layer (type) | Output Shape | Param # |
|------|------|------|
| Flatten | (None, 784) | 0 |
| Dense | (None, 1024) | 803840 |
| BatchNormalization | (None, 1024) | 4096 |
| Activation | (None, 1024) | 0 |
| Dense | (None, 1024) | 1049600 |
| BatchNormalization | (None, 1024) | 4096 |
| Activation | (None, 1024) | 0 |
| Dense | (None, 1024) | 1049600 |
| BatchNormalization | (None, 1024) | 4096 |
| Activation | (None, 1024) | 0 |
| Flatten | (None, 1024) | 0 |
| Dense | (None, 10) | 10250 |

In [21]:
bn_model = Sequential()
### START CODE HERE ###
bn_model.add(Flatten())
bn_model.add(Dense(1024))
bn_model.add(BatchNormalization())
bn_model.add(Activation('softmax'))
bn_model.add(Dense(1024))
bn_model.add(BatchNormalization())
bn_model.add(Activation('softmax'))
bn_model.add(Dense(1024))
bn_model.add(BatchNormalization())
bn_model.add(Activation('softmax'))
bn_model.add(Flatten())
bn_model.add(Dense(10, activation='softmax'))

bn_model.build(input_shape=(None, 28, 28, 1))

### END CODE HERE ###
bn_model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_12 (Flatten)        (None, 784)               0         
                                                                 
 dense_24 (Dense)            (None, 1024)              803840    
                                                                 
 batch_normalization_3 (Batc  (None, 1024)             4096      
 hNormalization)                                                 
                                                                 
 activation_3 (Activation)   (None, 1024)              0         
                                                                 
 dense_25 (Dense)            (None, 1024)              1049600   
                                                                 
 batch_normalization_4 (Batc  (None, 1024)             4096      
 hNormalization)                                      

BN Model을 학습해 보겠습니다. Generalization 성능이 올라갔나요?

In [22]:
bn_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])
bn_model_history=bn_model.fit(large_x_train, large_y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_validation, y_validation))

score = bn_model.evaluate(x_test, y_test, verbose=0)
print('BN model Test loss:', score[0])
print('BN model Test accuracy:', score[1])

Epoch 1/100
72/72 [==============================] - 4s 41ms/step - loss: 2.3026 - accuracy: 0.0900 - val_loss: 2.3027 - val_accuracy: 0.0700
Epoch 2/100
72/72 [==============================] - 3s 35ms/step - loss: 2.3026 - accuracy: 0.0900 - val_loss: 2.3027 - val_accuracy: 0.0700
Epoch 3/100
72/72 [==============================] - 3s 35ms/step - loss: 2.3026 - accuracy: 0.0900 - val_loss: 2.3027 - val_accuracy: 0.0700
Epoch 4/100
72/72 [==============================] - 3s 37ms/step - loss: 2.3026 - accuracy: 0.0900 - val_loss: 2.3027 - val_accuracy: 0.0700
Epoch 5/100
72/72 [==============================] - 3s 36ms/step - loss: 2.3026 - accuracy: 0.0900 - val_loss: 2.3027 - val_accuracy: 0.0700
Epoch 6/100
72/72 [==============================] - 3s 36ms/step - loss: 2.3026 - accuracy: 0.0900 - val_loss: 2.3027 - val_accuracy: 0.0700
Epoch 7/100
72/72 [==============================] - 3s 36ms/step - loss: 2.3026 - accuracy: 0.0900 - val_loss: 2.3027 - val_accuracy: 0.0700
Epoch 

##4.4 Final Model
지금까지 썼던 Regularization 기법들을 종합선물세트로 적용해 봅시다. 다음과 같은 Final model을 만들어 보세요.

| Layer (type) | Output Shape | Param # |
|------|------|------|
| Flatten | (None, 784) | 0 |
| Dense | (None, 1024) | 803840 |
| BatchNormalization | (None, 1024) | 4096 |
| Activation | (None, 1024) | 0 |
| Dense | (None, 1024) | 1049600 |
| BatchNormalization | (None, 1024) | 4096 |
| Activation | (None, 1024) | 0 |
| Dense | (None, 1024) | 1049600 |
| BatchNormalization | (None, 1024) | 4096 |
| Activation | (None, 1024) | 0 |
| Dropout | (None, 1024) | 0 |
| Flatten | (None, 1024) | 0 |
| Dense | (None, 10) | 10250 |

In [ ]:
final_model = Sequential()
### START CODE HERE ###

### END CODE HERE ###
final_model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_6 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_12 (Dense)             (None, 1024)              803840    
_________________________________________________________________
batch_normalization_3 (Batch (None, 1024)              4096      
_________________________________________________________________
activation_3 (Activation)    (None, 1024)              0         
_________________________________________________________________
dense_13 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_4 (Batch (None, 1024)              4096      
_________________________________________________________________
activation_4 (Activation)    (None, 1024)             

Final Model을 학습해 보겠습니다. Generalization 성능이 올라갔나요?

In [ ]:
final_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])
final_model_history=final_model.fit(large_x_train, large_y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_validation, y_validation))

score = final_model.evaluate(x_test, y_test, verbose=0)
print('Final model Test loss:', score[0])
print('Final model Test accuracy:', score[1])

Epoch 1/100
72/72 [==============================] - 4s 37ms/step - loss: 2.8955 - accuracy: 0.1204 - val_loss: 2.2754 - val_accuracy: 0.1200
Epoch 2/100
72/72 [==============================] - 2s 34ms/step - loss: 2.7084 - accuracy: 0.1313 - val_loss: 2.2247 - val_accuracy: 0.1550
Epoch 3/100
72/72 [==============================] - 2s 34ms/step - loss: 2.5755 - accuracy: 0.1531 - val_loss: 2.1182 - val_accuracy: 0.2250
Epoch 4/100
72/72 [==============================] - 2s 34ms/step - loss: 2.4187 - accuracy: 0.1860 - val_loss: 1.9820 - val_accuracy: 0.2850
Epoch 5/100
72/72 [==============================] - 2s 34ms/step - loss: 2.2413 - accuracy: 0.2345 - val_loss: 1.8511 - val_accuracy: 0.3250
Epoch 6/100
72/72 [==============================] - 2s 34ms/step - loss: 2.1545 - accuracy: 0.2505 - val_loss: 1.7301 - val_accuracy: 0.3950
Epoch 7/100
72/72 [==============================] - 2s 34ms/step - loss: 1.9716 - accuracy: 0.3018 - val_loss: 1.6215 - val_accuracy: 0.4500
Epoch 

(optional) Training data가 늘어나면 regularization 효과가 나는 것을 보였습니다. 쉽게 training data를 늘릴 수 있는 방법은 data augmentation 입니다. 이 방법은 기존 training data를 적절히 rotating, flipping, scaling, shifting 하여 training data 수를 늘리는 것입니다. data augmentation의 regularization 효과를 테스트해 보세요. 또한 뉴럴 네트워크의 노드 개수나 층수를 바꿔서 성능을 올려보는 것도 테스트해보세요.